In [280]:
from glob import glob
import pandas as pd
import numpy as np
from scipy import stats  
import matplotlib.pyplot as plt
import japanize_matplotlib
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [240]:
files = glob("../csv/演習問題_データアナリティクス基礎/*")
for i, f in enumerate(files):
    print(i, f)


0 ../csv/演習問題_データアナリティクス基礎\第10章演習問題_乱数で発生させて分布の基本統計量.xlsx
1 ../csv/演習問題_データアナリティクス基礎\第2章演習問題_クレームデータ.xlsx
2 ../csv/演習問題_データアナリティクス基礎\第3章演習問題_職場ストレス.xlsx
3 ../csv/演習問題_データアナリティクス基礎\第4章演習問題_児童のネット利用時間.xlsx
4 ../csv/演習問題_データアナリティクス基礎\第5章演習問題_生鮮野菜の購入額.xlsx
5 ../csv/演習問題_データアナリティクス基礎\第7章演習問題_授業効果の検証.xlsx
6 ../csv/演習問題_データアナリティクス基礎\第8章演習問題1_健康食品・化粧品販売額.xlsx
7 ../csv/演習問題_データアナリティクス基礎\第8章演習問題2_百貨店の衣料品売上高.xlsx
8 ../csv/演習問題_データアナリティクス基礎\第9章演習問題_自由記述データ＆形態素解析データ.xlsx


In [231]:
df = pd.read_excel(files[7])
print(df.columns)
df.columns = ['date', 'total', 'men', 'women', 'child', 'other']
df.info()
df.head()

Index(['Unnamed: 0', '計', '紳士服', '婦人服', '子供服', 'その他\n衣料品'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    25 non-null     object 
 1   total   25 non-null     float64
 2   men     25 non-null     float64
 3   women   25 non-null     float64
 4   child   25 non-null     float64
 5   other   25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.3+ KB


,date,total,men,women,child,other
0,40360,32264.2814,6435.1631,20564.8712,2325.2873,2938.9598
1,8月,19807.8368,3300.1486,12929.5010,1369.2929,2208.8943
2,9月,25361.5249,4081.2346,17153.2525,1745.9854,2381.0524
3,10月,33025.8470,6607.9540,21257.7765,2290.9460,2869.1705
4,11月,31468.2275,6981.3833,19685.4091,1986.5516,2814.8835


月別の平均の最大値はいつか

In [232]:
callender = []
start_date = 2010, 7, 1
for i in range(df.shape[0]):
    s_date = date(start_date[0], start_date[1], start_date[2])
    adjust = s_date + relativedelta(months=i+1) - relativedelta(days=1)
    callender.append(adjust)
df['date'] = pd.to_datetime(callender)
df['month'] = df['date'].dt.strftime('%m')
df = df.set_index('date')

In [233]:
monthly_mean = {}
for i in range(12):
    monthly_mean[f'{i+1}月平均'] = round(df.query(f'month == "{i+1:02}"')['child'].mean(), 2)
monthly_mean

{'1月平均': 2537.39,
 '2月平均': 1687.52,
 '3月平均': 3469.89,
 '4月平均': 2448.71,
 '5月平均': 2063.86,
 '6月平均': 1551.09,
 '7月平均': 2207.4,
 '8月平均': 1361.78,
 '9月平均': 1890.16,
 '10月平均': 2217.47,
 '11月平均': 2004.01,
 '12月平均': 2344.43}

婦人服の2017年7月基準の指数

In [234]:
df['women'] / df.loc['2010-07-31', 'women'] * 100

date
2010-07-31    100.000000
2010-08-31     62.871782
2010-09-30     83.410454
2010-10-31    103.369364
2010-11-30     95.723474
2010-12-31    100.268456
2011-01-31    105.527579
2011-02-28     68.209732
2011-03-31     99.400438
2011-04-30     89.747682
2011-05-31     86.239860
2011-06-30     76.810335
2011-07-31     95.600114
2011-08-31     60.192296
2011-09-30     84.649641
2011-10-31     99.511743
2011-11-30     96.608699
2011-12-31     94.296211
2012-01-31    105.775597
2012-02-29     63.866756
2012-03-31    102.600167
2012-04-30     87.930009
2012-05-31     83.548073
2012-06-30     75.751411
2012-07-31     92.021500
Name: women, dtype: float64

紳士服の2011年7月対前年同月比での増加率

In [235]:
incrude_rate = (df.loc['2011-07-31', 'men'] - df.loc['2010-07-31', 'men']) / df.loc['2010-07-31', 'men'] * 100
print(f'{incrude_rate:.01f}')

-8.3


2011年8月以降で衣料品合計の対前年度同月比、最大減少率の月はいつか

In [236]:
df.index[13:]

DatetimeIndex(['2011-08-31', '2011-09-30', '2011-10-31', '2011-11-30',
               '2011-12-31', '2012-01-31', '2012-02-29', '2012-03-31',
               '2012-04-30', '2012-05-31', '2012-06-30', '2012-07-31'],
              dtype='datetime64[ns]', name='date', freq=None)

In [241]:
df_yoy = pd.DataFrame([df.loc['2011-8':]['total'].values, df.loc['2010-8':'2011-7']['total'].values]).T
df_yoy.columns = ['comparison', 'reference']
df_yoy['increase_rete'] = (df_yoy['comparison'] - df_yoy['reference']) / df_yoy['reference'] * 100
df_yoy.index = df.index[13:]
df_yoy.loc[df_yoy['increase_rete'].idxmax(), 'mark'] = 'max'
df_yoy.loc[df_yoy['increase_rete'].idxmin(), 'mark'] = 'min'
df_yoy

,comparison,reference,increase_rete,mark
date,,,,
2011-08-31,18841.9573,19807.8368,-4.876249,NaN
2011-09-30,25929.7685,25361.5249,2.240573,max
2011-10-31,31337.9934,33025.8470,-5.110705,NaN
2011-11-30,31299.9116,31468.2275,-0.534876,NaN
2011-12-31,32989.9032,35504.5797,-7.082682,NaN
2012-01-31,33107.4060,33690.6188,-1.731084,NaN
2012-02-29,20610.8787,22236.8845,-7.312201,min
2012-03-31,32436.5192,31991.6711,1.390512,NaN
2012-04-30,27982.1969,28994.9121,-3.492734,NaN


各売上に対し回帰直線を作り下降トレンドでないものを示す

In [295]:
# scikit-learn の LinearRegression
y_train = [i+1 for i in range(df.shape[0])]
model_lr = LinearRegression()
for c in df.columns[1:-1]:
    x_train = df[[c]]
    model_lr.fit(x_train, y_train)
    print(f'{c}の傾き: {model_lr.coef_[0]:.05f}')

menの傾き: -0.00035
womenの傾き: -0.00017
childの傾き: 0.00095
otherの傾き: -0.00706


In [302]:
# statsmodels の 線形回帰
for c in df.columns[1:-1]:
    x_train = df[[c]]
    y_train = [i+1 for i in range(df.shape[0])]
    x_add_const = sm.add_constant(x_train)
    model_sm = sm.OLS(y_train, x_add_const).fit()
    print(f'{c}の傾き: {model_sm.params[1]:.05f}')

menの傾き: -0.00035
womenの傾き: -0.00017
childの傾き: 0.00095
otherの傾き: -0.00706
